### Задачи к Лекции 4

__Исходные данные__ 

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]


df = pd.read_csv(r"C:\Users\Kirill\OneDrive\Desktop\study\Технологии автоматической обработки текста\lecture 4\mlbootcamp5_train.csv", 
                 sep=";", 
                 index_col="id")
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,True,False,False,True,False,False,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,False,False,True,True,False,False,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,False,False,True,True,False,False,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,True,False,False,True,False,False,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,True,False,False,True,False,False,0


## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедент.

In [9]:
# A lot of code here
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.model_selection import KFold

In [10]:
class KNNClassifier:
    def __init__(self, k):
        self.k = k  # Число ближайших соседей
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        """Сохраняем обучающую выборку."""
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        """Предсказываем метки для тестовой выборки."""
        predictions = []
        for x in X_test:
            # Вычисляем расстояния от текущего объекта до всех объектов в обучающей выборке
            distances = np.linalg.norm(self.X_train - x, axis=1)
            
            # Находим индексы k ближайших соседей
            nearest_indices = np.argsort(distances)[:self.k]
            
            # Получаем метки ближайших соседей
            nearest_labels = self.y_train[nearest_indices]
            
            # Проводим голосование (выбираем наиболее часто встречающуюся метку)
            prediction = np.bincount(nearest_labels).argmax()
            predictions.append(prediction)
        
        return np.array(predictions)

In [11]:
# Выбор признаков и целевой переменной
X = df[["gender", "cholesterol", "gluc"]].values  # Преобразуем в массив numpy
y = df["cardio"].values

In [12]:
# Функция для кросс-валидации
def cross_validate_knn(X, y, k_values, n_folds=5):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    best_k = None
    best_accuracy = 0

    for k in k_values:
        accuracies = []
        for train_index, test_index in kf.split(X):
            # Разделение данных на обучающую и тестовую выборки
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # Обучение и предсказание
            knn = KNNClassifier(k=k)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)

            # Оценка точности
            accuracy = np.mean(y_pred == y_test)
            accuracies.append(accuracy)

        # Средняя точность для данного k
        mean_accuracy = np.mean(accuracies)
        print(f"k = {k}, средняя точность = {mean_accuracy:.4f}")

        # Обновление лучшего значения k
        if mean_accuracy > best_accuracy:
            best_accuracy = mean_accuracy
            best_k = k

    return best_k, best_accuracy

# Диапазон значений k для проверки
k_values = range(1, 11)

# Поиск оптимального k
best_k, best_accuracy = cross_validate_knn(X, y, k_values)
print(f"Оптимальное значение k: {best_k}")
print(f"Наивысшая точность: {best_accuracy:.4f}")

KeyboardInterrupt: 

In [ ]:
# Обучение модели с оптимальным k
optimal_knn = KNNClassifier(k=best_k)
optimal_knn.fit(X, y)

# Предсказание на исходных данных
y_pred = optimal_knn.predict(X)

# Матрица неточностей
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y, y_pred)
print("Матрица неточностей:")
print(cm)

# Точность
accuracy = np.mean(y == y_pred)
print(f"Точность на обучающей выборке: {accuracy:.4f}")

Матрица неточностей:
[[32322  2699]
 [28154  6825]]
Точность на обучающей выборке: 0.5592


**Комментарии:** Ваши комментарии здесь.

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [15]:
# Your code here
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Выбор признаков и целевой переменной
X = df[['age', 'weight', 'height', 'ap_lo', 'ap_hi']]
y = df['cardio']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение kNN
knn = KNeighborsClassifier(n_neighbors=5)  # Оптимальное k можно найти через кросс-валидацию
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
y_pred_proba_knn = knn.predict_proba(X_test)[:, 1]

# Обучение наивного Байеса
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
y_pred_proba_nb = nb.predict_proba(X_test)[:, 1]

# Обучение решающего дерева
tree = DecisionTreeClassifier(max_depth=5, random_state=42)  # Оптимальная глубина может быть найдена через кросс-валидацию
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)
y_pred_proba_tree = tree.predict_proba(X_test)[:, 1]

def evaluate_metrics(y_true, y_pred, y_pred_proba):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    loss = log_loss(y_true, y_pred_proba)
    return accuracy, f1, roc_auc, loss

# Метрики для kNN
knn_metrics = evaluate_metrics(y_test, y_pred_knn, y_pred_proba_knn)

# Метрики для наивного Байеса
nb_metrics = evaluate_metrics(y_test, y_pred_nb, y_pred_proba_nb)

# Метрики для решающего дерева
tree_metrics = evaluate_metrics(y_test, y_pred_tree, y_pred_proba_tree)

# Вывод результатов
metrics_names = ['Accuracy', 'F1-score', 'ROC AUC', 'Log Loss']
results = {
    'kNN': knn_metrics,
    'Naive Bayes': nb_metrics,
    'Decision Tree': tree_metrics
}

# Создаем таблицу для сравнения
comparison_df = pd.DataFrame(results, index=metrics_names)
print(comparison_df)

               kNN  Naive Bayes  Decision Tree
Accuracy  0.681071     0.562429       0.724143
F1-score  0.677221     0.304496       0.713671
ROC AUC   0.728307     0.682941       0.785521
Log Loss  2.354811     0.721895       0.561885


**Комментарии:** Ваши комментарии здесь.